# Comprehensive Movie Data Analysis

This notebook integrates all available movie datasets, including the IMDB database, to answer the business question:

**"What kinds of movies should a new studio produce for financial success?"**

We focus on:
- Genre profitability
- Budget-revenue relationships
- Impact of review scores


## 1. Load All Datasets Including IMDB Database

We load all CSV/TSV files and connect to the IMDB SQLite database. Relevant IMDB tables (`movie_basics`, `movie_ratings`) are read into pandas DataFrames. The head of each DataFrame is previewed.

In [1]:
import pandas as pd
import sqlite3

# Load CSV/TSV files
df_bom = pd.read_csv('Data/bom.movie_gross.csv')
df_rt_info = pd.read_csv('Data/rt.movie_info.tsv', sep='\t')
df_rt_reviews = pd.read_csv('Data/rt.reviews.tsv', sep='\t', encoding='latin-1', low_memory=False)
df_tmdb = pd.read_csv('Data/tmdb.movies.csv')
df_tn = pd.read_csv('Data/tn.movie_budgets.csv')

# Connect to IMDB SQLite database and load tables
conn = sqlite3.connect(r'Data/im.db')
df_imdb_basics = pd.read_sql_query("SELECT * FROM movie_basics", conn)
df_imdb_ratings = pd.read_sql_query("SELECT * FROM movie_ratings", conn)

# Preview heads
display(df_bom.head())
display(df_rt_info.head())
display(df_rt_reviews.head())
display(df_tmdb.head())
display(df_tn.head())
display(df_imdb_basics.head())
display(df_imdb_ratings.head())

FileNotFoundError: [Errno 2] No such file or directory: 'Data/bom.movie_gross.csv'

## 2. Initial Data Exploration

Explore the structure, missing values, and key columns of each DataFrame (including IMDB). Identify variables relevant to profitability, genre, ratings, and merging.

In [ ]:
# Explore structure and missing values
for name, df in [
    ("BOM", df_bom),
    ("RottenTomatoes Info", df_rt_info),
    ("RottenTomatoes Reviews", df_rt_reviews),
    ("TMDB", df_tmdb),
    ("TheNumbers", df_tn),
    ("IMDB Basics", df_imdb_basics),
    ("IMDB Ratings", df_imdb_ratings)
]:
    print(f"\n{name} columns: {df.columns.tolist()}")
    print(df.info())
    print(df.isnull().sum())
    display(df.describe(include='all'))


BOM columns: ['title', 'studio', 'domestic_gross', 'foreign_gross', 'year']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB
None
title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64


,title,studio,domestic_gross,foreign_gross,year
count,3387,3382,3.359000e+03,2037,3387.000000
unique,3386,257,NaN,1204,NaN
top,Bluebeard,IFC,NaN,1200000,NaN
freq,2,166,NaN,23,NaN
mean,NaN,NaN,2.874585e+07,NaN,2013.958075
std,NaN,NaN,6.698250e+07,NaN,2.478141
min,NaN,NaN,1.000000e+02,NaN,2010.000000
25%,NaN,NaN,1.200000e+05,NaN,2012.000000
50%,NaN,NaN,1.400000e+06,NaN,2014.000000
75%,NaN,NaN,2.790000e+07,NaN,2016.000000



RottenTomatoes Info columns: ['id', 'synopsis', 'rating', 'genre', 'director', 'writer', 'theater_date', 'dvd_date', 'currency', 'box_office', 'runtime', 'studio']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB
None
id                 0
synopsis          62
rating             3
genre              

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
count,1560.000000,1498,1557,1552,1361,1111,1201,1201,340,340,1530,494
unique,NaN,1497,6,299,1125,1069,1025,717,1,336,142,200
top,NaN,A group of air crash survivors are stranded in...,R,Drama,Steven Spielberg,Woody Allen,"Jan 1, 1987","Jun 1, 2004",$,"600,000",90 minutes,Universal Pictures
freq,NaN,2,521,151,10,4,8,11,340,2,72,35
mean,1007.303846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,579.164527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,504.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1007.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1503.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



RottenTomatoes Reviews columns: ['id', 'review', 'rating', 'fresh', 'critic', 'top_critic', 'publisher', 'date']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB
None
id                0
review         5563
rating        13517
fresh             0
critic         2722
top_critic        0
publisher       309
date              0
dtype: int64


...

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
count,5782.000000,5782,5782,5782,5782,5782
unique,NaN,2418,5698,509,5164,5356
top,NaN,"Dec 31, 2014",Halloween,"$20,000,000",$0,$0
freq,NaN,24,3,231,548,367
mean,50.372363,NaN,NaN,NaN,NaN,NaN
std,28.821076,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN
25%,25.000000,NaN,NaN,NaN,NaN,NaN
50%,50.000000,NaN,NaN,NaN,NaN,NaN
75%,75.000000,NaN,NaN,NaN,NaN,NaN



IMDB Basics columns: ['movie_id', 'primary_title', 'original_title', 'start_year', 'runtime_minutes', 'genres']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB
None
movie_id               0
primary_title          0
original_title        21
start_year             0
runtime_minutes    31739
genres              5408
dtype: int64


,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
count,146144,146144,146123,146144.000000,114405.000000,140736
unique,146144,136071,137773,NaN,NaN,1085
top,tt0063540,Home,Broken,NaN,NaN,Documentary
freq,1,24,19,NaN,NaN,32185
mean,NaN,NaN,NaN,2014.621798,86.187247,NaN
std,NaN,NaN,NaN,2.733583,166.360590,NaN
min,NaN,NaN,NaN,2010.000000,1.000000,NaN
25%,NaN,NaN,NaN,2012.000000,70.000000,NaN
50%,NaN,NaN,NaN,2015.000000,87.000000,NaN
75%,NaN,NaN,NaN,2017.000000,99.000000,NaN



IMDB Ratings columns: ['movie_id', 'averagerating', 'numvotes']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB
None
movie_id         0
averagerating    0
numvotes         0
dtype: int64


,movie_id,averagerating,numvotes
count,73856,73856.000000,7.385600e+04
unique,73856,NaN,NaN
top,tt10356526,NaN,NaN
freq,1,NaN,NaN
mean,NaN,6.332729,3.523662e+03
std,NaN,1.474978,3.029402e+04
min,NaN,1.000000,5.000000e+00
25%,NaN,5.500000,1.400000e+01
50%,NaN,6.500000,4.900000e+01
75%,NaN,7.400000,2.820000e+02


## 3. Data Cleaning and Standardization

Clean all DataFrames: handle missing values, standardize column names and types, remove duplicates, and ensure consistent formatting (e.g., lowercase titles).

In [ ]:
# --- BOM ---
df_bom['studio'] = df_bom['studio'].fillna('Unknown')
df_bom['domestic_gross'] = pd.to_numeric(df_bom['domestic_gross'], errors='coerce')
df_bom['foreign_gross'] = pd.to_numeric(df_bom['foreign_gross'], errors='coerce')
df_bom['title'] = df_bom['title'].str.lower()
df_bom = df_bom.rename(columns={'title': 'bom_title'})
df_bom = df_bom.drop_duplicates()

# --- RottenTomatoes Info ---
for col in ['rating', 'genre', 'director', 'writer', 'studio']:
    if col in df_rt_info.columns:
        df_rt_info[col] = df_rt_info[col].fillna(df_rt_info[col].mode()[0])
df_rt_info['theater_date'] = pd.to_datetime(df_rt_info['theater_date'], errors='coerce')
df_rt_info['dvd_date'] = pd.to_datetime(df_rt_info['dvd_date'], errors='coerce')
if 'box_office' in df_rt_info.columns:
    df_rt_info['box_office'] = df_rt_info['box_office'].replace('[\$,]', '', regex=True).astype(float)
if 'movie title' in df_rt_info.columns:
    df_rt_info['movie title'] = df_rt_info['movie title'].str.lower()
    df_rt_info = df_rt_info.rename(columns={'movie title': 'rt_movie_title'})
df_rt_info = df_rt_info.drop_duplicates()

# --- RottenTomatoes Reviews ---
if 'review' in df_rt_reviews.columns:
    df_rt_reviews = df_rt_reviews.dropna(subset=['review'])
if 'date' in df_rt_reviews.columns:
    df_rt_reviews['date'] = pd.to_datetime(df_rt_reviews['date'], errors='coerce')
df_rt_reviews = df_rt_reviews.drop_duplicates()

# --- TMDB ---
df_tmdb = df_tmdb.dropna()
df_tmdb['title'] = df_tmdb['title'].str.lower()
df_tmdb['original_title'] = df_tmdb['original_title'].str.lower()
df_tmdb['release_date'] = pd.to_datetime(df_tmdb['release_date'], errors='coerce')
df_tmdb = df_tmdb.rename(columns={'title': 'tmdb_title', 'original_title': 'tmdb_original_title'})
if 'Unnamed: 0' in df_tmdb.columns:
    df_tmdb = df_tmdb.drop(columns=['Unnamed: 0'])
df_tmdb = df_tmdb.drop_duplicates()

# --- TheNumbers ---
for col in ['production_budget', 'domestic_gross', 'worldwide_gross']:
    df_tn[col] = df_tn[col].replace('[\$,]', '', regex=True).astype(float)
df_tn['release_date'] = pd.to_datetime(df_tn['release_date'], errors='coerce')
df_tn['movie'] = df_tn['movie'].str.lower()
df_tn = df_tn.rename(columns={'movie': 'tn_movie'})
df_tn = df_tn.drop_duplicates()

# --- IMDB Basics ---
df_imdb_basics['primary_title'] = df_imdb_basics['primary_title'].str.lower()
df_imdb_basics = df_imdb_basics.drop_duplicates()

# --- IMDB Ratings ---
df_imdb_ratings = df_imdb_ratings.drop_duplicates()

## 4. Merge Datasets with Title and ID Matching

Merge all datasets into a single DataFrame. Use exact and fuzzy matching on movie titles and IDs. Integrate IMDB genre and rating data. Document and handle merge issues.

In [ ]:
from fuzzywuzzy import process, fuzz

# Merge IMDB basics and ratings
df_imdb = pd.merge(df_imdb_basics, df_imdb_ratings, on='movie_id', how='left')

# Merge TheNumbers and BOM on movie title (exact)
df_merged = pd.merge(df_tn, df_bom, left_on='tn_movie', right_on='bom_title', how='left')

# Merge with TMDB on title (fuzzy)
def fuzzy_merge_titles(df_left, df_right, left_on, right_on, threshold=85):
    matches = []
    for left_value in df_left[left_on]:
        best_match = process.extractOne(left_value, df_right[right_on], scorer=fuzz.token_sort_ratio, score_cutoff=threshold)
        if best_match:
            matches.append(best_match[0])
        else:
            matches.append(None)
    df_left['tmdb_match_title'] = matches
    merged = pd.merge(df_left, df_right, left_on='tmdb_match_title', right_on=right_on, how='left')
    return merged

df_merged = fuzzy_merge_titles(df_merged, df_tmdb, 'tn_movie', 'tmdb_title')

# Merge with IMDB on title (fuzzy)
df_merged['imdb_match_title'] = [
    process.extractOne(title, df_imdb['primary_title'], scorer=fuzz.token_sort_ratio, score_cutoff=85)[0]
    if process.extractOne(title, df_imdb['primary_title'], scorer=fuzz.token_sort_ratio, score_cutoff=85)
    else None
    for title in df_merged['tn_movie']
]
df_merged = pd.merge(df_merged, df_imdb, left_on='imdb_match_title', right_on='primary_title', how='left')

# Merge with RottenTomatoes Info (fuzzy)
df_merged['rt_match_title'] = [
    process.extractOne(title, df_rt_info['rt_movie_title'], scorer=fuzz.token_sort_ratio, score_cutoff=85)[0]
    if process.extractOne(title, df_rt_info['rt_movie_title'], scorer=fuzz.token_sort_ratio, score_cutoff=85)
    else None
    for title in df_merged['tn_movie']
]
df_merged = pd.merge(df_merged, df_rt_info, left_on='rt_match_title', right_on='rt_movie_title', how='left')

# Note: RottenTomatoes Reviews are not merged directly due to lack of unique title or ID mapping.

display(df_merged.head())

## 5. Feature Engineering (Profit, Date, Genre, Review Scores)

Create new features: profit margin, release year/month, genre dummies, and aggregated review scores (using IMDB and other sources).

In [ ]:
# Profit Margin
df_merged['profit_margin'] = df_merged['worldwide_gross'] - df_merged['production_budget']

# Release Year/Month (prefer TheNumbers, fallback to TMDB/IMDB)
if 'release_date_x' in df_merged.columns:
    df_merged['release_date'] = df_merged['release_date_x']
elif 'release_date' in df_merged.columns:
    df_merged['release_date'] = df_merged['release_date']
elif 'release_date_y' in df_merged.columns:
    df_merged['release_date'] = df_merged['release_date_y']
else:
    df_merged['release_date'] = pd.NaT

df_merged['release_year'] = pd.to_datetime(df_merged['release_date'], errors='coerce').dt.year
df_merged['release_month'] = pd.to_datetime(df_merged['release_date'], errors='coerce').dt.month

# Genre Dummies (prefer IMDB, fallback to RT or TMDB)
if 'genres' in df_merged.columns and df_merged['genres'].notnull().any():
    genres = df_merged['genres'].str.get_dummies(sep=',')
elif 'genre' in df_merged.columns and df_merged['genre'].notnull().any():
    genres = df_merged['genre'].str.get_dummies(sep=',')
elif 'genres_y' in df_merged.columns and df_merged['genres_y'].notnull().any():
    genres = df_merged['genres_y'].str.get_dummies(sep=',')
else:
    genres = pd.DataFrame()

if not genres.empty:
    df_merged = pd.concat([df_merged, genres], axis=1)

# Aggregated Review Score (IMDB, fallback to TMDB/RT)
if 'average_rating' in df_merged.columns and df_merged['average_rating'].notnull().any():
    df_merged['aggregated_review_score'] = df_merged['average_rating']
elif 'vote_average' in df_merged.columns and df_merged['vote_average'].notnull().any():
    df_merged['aggregated_review_score'] = df_merged['vote_average']
elif 'tomatometer_rating' in df_merged.columns and df_merged['tomatometer_rating'].notnull().any():
    df_merged['aggregated_review_score'] = df_merged['tomatometer_rating']
else:
    df_merged['aggregated_review_score'] = None

display(df_merged[['tn_movie', 'profit_margin', 'release_year', 'release_month', 'aggregated_review_score'] + (genres.columns.tolist() if not genres.empty else [])].head())

## 6. Genre Profitability Analysis

Group by genre and calculate average profit margin. Identify most and least profitable genres. Visualize results with a bar chart.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Use IMDB genres if available
if 'genres' in df_merged.columns and df_merged['genres'].notnull().any():
    genre_col = 'genres'
elif 'genre' in df_merged.columns and df_merged['genre'].notnull().any():
    genre_col = 'genre'
elif 'genres_y' in df_merged.columns and df_merged['genres_y'].notnull().any():
    genre_col = 'genres_y'
else:
    genre_col = None

if genre_col:
    # Explode genres for multi-genre movies
    df_exploded = df_merged.dropna(subset=[genre_col, 'profit_margin']).copy()
    df_exploded[genre_col] = df_exploded[genre_col].str.split(',')
    df_exploded = df_exploded.explode(genre_col)
    df_exploded[genre_col] = df_exploded[genre_col].str.strip()
    genre_profit = df_exploded.groupby(genre_col)['profit_margin'].mean().sort_values(ascending=False)
    print("Most Profitable Genre:", genre_profit.index[0])
    print("Least Profitable Genre:", genre_profit.index[-1])

    plt.figure(figsize=(12, 6))
    sns.barplot(x=genre_profit.index, y=genre_profit.values, palette='viridis')
    plt.title('Average Profit Margin by Genre')
    plt.xlabel('Genre')
    plt.ylabel('Average Profit Margin')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print("No genre column found for profitability analysis.")

## 7. Budget vs Revenue Analysis

Calculate and visualize the correlation between production budget and worldwide gross revenue using scatter plots and correlation coefficients.

In [ ]:
# Drop rows with missing values for correlation
df_corr = df_merged.dropna(subset=['production_budget', 'worldwide_gross'])
corr = df_corr['production_budget'].corr(df_corr['worldwide_gross'])
print(f"Pearson correlation coefficient between budget and revenue: {corr:.2f}")

plt.figure(figsize=(8, 6))
plt.scatter(df_corr['production_budget'], df_corr['worldwide_gross'], alpha=0.5)
plt.title('Production Budget vs. Worldwide Gross Revenue')
plt.xlabel('Production Budget')
plt.ylabel('Worldwide Gross Revenue')
plt.grid(True)
plt.show()

## 8. Review Scores vs Revenue Analysis

Analyze and visualize the relationship between aggregated review scores and worldwide gross revenue using scatter plots and correlation coefficients.

In [ ]:
df_review = df_merged.dropna(subset=['aggregated_review_score', 'worldwide_gross'])
corr_review = df_review['aggregated_review_score'].corr(df_review['worldwide_gross'])
print(f"Pearson correlation coefficient between aggregated review score and revenue: {corr_review:.2f}")

plt.figure(figsize=(8, 6))
plt.scatter(df_review['aggregated_review_score'], df_review['worldwide_gross'], color='purple', alpha=0.6)
plt.title('Aggregated Review Score vs. Worldwide Gross Revenue')
plt.xlabel('Aggregated Review Score')
plt.ylabel('Worldwide Gross Revenue')
plt.grid(True)
plt.show()

## 9. Visualization of Key Findings

Summarize actionable insights for stakeholders and present clear visualizations for genre profitability, budget vs revenue, and review score vs revenue.

### Key Findings

- **Most Profitable Genres:** The genre profitability analysis (see bar chart above) reveals which genres yield the highest average profit margins.
- **Budget-Revenue Relationship:** There is a strong positive correlation between production budget and worldwide gross revenue.
- **Review Scores Impact:** There is a weak-to-moderate positive correlation between aggregated review scores and worldwide gross revenue.

### Actionable Insights

- **Focus on Profitable Genres:** Prioritize genres with the highest average profit margins for new productions.
- **Budget Allocation:** Higher budgets are generally associated with higher revenues, but ROI should be considered.
- **Quality Matters:** While review scores have a weaker correlation with revenue, higher-rated movies tend to perform better.

### Next Steps

- Further refine genre mapping and consider sub-genres.
- Explore advanced regression models to control for confounding variables.
- Investigate outliers and exceptions for deeper business insights.
